In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
from sklearn.utils import shuffle
from torch.utils.data import DataLoader, Dataset, TensorDataset

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
class MyDataset(Dataset):
    def __init__(self, csv_file, device):
        self.data = pd.read_csv(csv_file)
        self.device = device
        # Add any necessary data preprocessing or feature extraction here

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {
            'data': torch.tensor(self.data.iloc[idx, 1:].values, dtype=torch.float64, device=self.device),
            'label': torch.tensor(self.data.iloc[idx, 0], dtype=torch.long, device=self.device)
        }
        return sample

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train = MyDataset("../input/fashionmnist/fashion-mnist_train.csv", device)
test = MyDataset("../input/fashionmnist/fashion-mnist_test.csv", device)

In [ ]:
train_label = []
train_data = []
for i in range(len(train)):
    sample = train[i]
    train_label.append(sample['label'])
    train_data.append(sample['data'])

test_label = []
test_data = []
for i in range(len(test)):
    sample = test[i]
    test_label.append(sample['label'])
    test_data.append(sample['data'])

In [ ]:
def create_batches(data, label):
    num_data = len(data)
    num_batches = num_data // 32
    batches_x = []
    batches_y = []
    shuffled_indices = np.random.permutation(num_data)
    
    for i in range(0, num_data, num_batches):
        batch_indices = shuffled_indices[i:i+32]
        batch_x = torch.stack([data[idx] for idx in batch_indices]).to(device)
#         batch_x = batch_x.double()
        batch_y = torch.stack([label[idx] for idx in batch_indices]).to(device)
#         batch_y = batch_y.double()
        batches_x.append(batch_x)
        batches_y.append(batch_y)
    return batches_x, batches_y

In [ ]:
class Model(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        out = self.net(x)
        return out

In [ ]:
def calculate_loss(out, y):
    y_pred = torch.argmax(out)
    return (y_pred == y)

In [ ]:
model = Model(device)
# model = model.double()
model.to(device)
learning_rate = 1e-3
n_epochs = 10
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate) 
criterion = nn.CrossEntropyLoss()

In [ ]:
train_data_tensor = torch.stack(train_data)
train_label_tensor = torch.stack(train_label)

print(train_data_tensor.shape)
print(train_label_tensor.shape)

In [ ]:
X, Y = create_batches(train_data, train_label)
data_loader = DataLoader(train, batch_size=32, shuffle=True)

In [ ]:
start_time = time.time()

for epoch in range(n_epochs):
    idx = 0
    for batch in data_loader:
        data = batch[0]
        label = batch[1]
        out = model.forward(data.to(model.net[0].weight.dtype))
#         loss = criterion(out, batch_y.to(torch.long))
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (idx % 100 == 0):
            t = int(time.time() - start_time); 
            t_str = '%2dh %2dm %2ds' % (int(t / 3600), int((t % 3600) / 60), t % 60)
            print(f"Time: {t_str} step {idx}: loss: {loss:.4f}")
        idx += 1
    print(f"Epoch: {epoch+1} complete")

In [ ]:
model.eval()
i = 0 
correct = 0
softmax = nn.Softmax()
for batch_x, batch_y in zip(X, Y):
    out = model.forward(batch_x.to(model.net[0].weight.dtype))
    a = softmax(out)
    loss = criterion(out, batch_y.to(torch.long))
    for j in range(out.shape[0]):
        result = torch.argmax(a[j,:])
        correct += (result == batch_y[j])
    
    if (i % 10 == 0):
        print(f"Iter: {i+1} loss: {loss} accuracy: {correct/(32*(i+1))}")
        
    i += 1
model.train()

In [ ]:
XV, YV = create_batches(test_data, test_label)

In [ ]:
model.eval()
i = 0 
correct = 0
softmax = nn.Softmax()
for batch_x, batch_y in zip(XV, YV):
    out = model.forward(batch_x.to(model.net[0].weight.dtype))
    a = softmax(out)
    loss = criterion(out, batch_y.to(torch.long))
    for j in range(out.shape[0]):
        result = torch.argmax(a[j,:])
        correct += (result == batch_y[j])
    
    if (i % 10 == 0):
        print(f"Iter: {i+1} loss: {loss} accuracy: {correct/(32*(i+1))}")
        
    i += 1
model.train()